In [1]:
import json
from pathlib import Path
from dataset import *
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from model import *
from tqdm import tqdm
import sys
import os
from metrics import *
import torch
import argparse


import pynvml
pynvml.nvmlInit()
handle = pynvml.nvmlDeviceGetHandleByIndex(0)   #gpu_id
meminfo = pynvml.nvmlDeviceGetMemoryInfo(handle)


2022-07-13 21:25:43.201732: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
if not os.path.exists("./outputs"):
    os.mkdir("./outputs")
data_dir = Path('..//input/')

train_mark_path = './data/train_mark.csv'
train_features_path = "./data/train_fts.json"
val_mark_path = './data/val_mark.csv'
val_features_path = './data/val_fts.json'
val_path = "./data/val.csv"
model_name_or_path ="microsoft/codebert-base"

md_max_len = 512
total_max_len = 64
batch_size= 1
accumulation_steps=1
epochs=2 
n_workers=4

In [3]:
train_df_mark = pd.read_csv(train_mark_path).drop("parent_id", axis=1).dropna().reset_index(drop=True)
train_fts = json.load(open(train_features_path))
val_df_mark = pd.read_csv(val_mark_path).drop("parent_id", axis=1).dropna().reset_index(drop=True)
val_fts = json.load(open(val_features_path))
val_df = pd.read_csv(val_path)

order_df = pd.read_csv("../input/train_orders.csv").set_index("id")
#df_orders = pd.read_csv(
#    data_dir / 'train_orders.csv',
#    index_col='id',
#    squeeze=True,
#).str.split()

order = pd.read_csv(
    data_dir / 'train_orders.csv'
)
train_path = './data/train.csv'
val_path = './data/val.csv'
val_df = pd.read_csv(val_path)
train_df = pd.read_csv(train_path)

full_df = pd.concat([train_df, val_df])

In [4]:
# tmp = train_df_mark[train_df_mark['cell_type']=='markdown'].groupby("id").count()['cell_id']
# tmp.apply(lambda x: x**2).sum()/len(tmp) # = 539
# tmp.mean() # = 15.56
# train_df.head()

In [5]:
def get_dict(df):
    md_dict = dict(zip(df["cell_id"].values, df['source'].values))
    return md_dict

md_dict = get_dict(full_df[full_df['cell_type']=='markdown'])
cd_dict = get_dict(full_df[full_df['cell_type']=='code'])

In [6]:
#len(md_dict), len(cd_dict)
#len(train_df_mark), len(train_df), len(val_df_mark), len(val_df)
#order.head()

In [7]:
%time
x = 1<<32
seed = 42
def transform(row):
    
    cell_ids = list(row["cell_order"].split())
    md_ids = []
    cell_shuffle = []
    md_mask = []
    for cell_id in cell_ids:
        if cell_id in md_dict:
            cell_shuffle.append(0)
            md_ids.append(cell_id)
            md_mask.append(1)
        else:
            cell_shuffle.append(cell_id)
            md_mask.append(0)
    length = len(md_ids)
    _hash = hash(row["cell_order"])*seed % x
    permutation = np.arange(length)
    #permutation = np.random.RandomState(seed=_hash).permutation(length)
    i = 0
    for j in range(len(cell_shuffle)):
        if cell_shuffle[j] == 0:
            cell_shuffle[j] = cell_ids[permutation[i]]
            i+=1
    
    return pd.Series([row.id, cell_shuffle, permutation, md_mask], index=['id', 'cell_shuffle', 'permutation', 'md_mask'])

order = order.apply(transform, axis=1)
order.head()

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 2.62 µs


,id,cell_shuffle,permutation,md_mask
0,00001756c60be8,"[1862f0a6, 1862f0a6, 2a9e43d6, 448eb224, 038b7...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, ..."
1,00015c83e2717b,"[2e94bd7a, 3e99dee9, b5e286ea, da4f7550, c4172...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, ..."
2,0001bdd4021779,"[3fdc37be, 073782ca, 8ea7263c, 80543cd8, 38310...","[0, 1]","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1]"
3,0001daf4c2c76d,"[97266564, a898e555, 86605076, 86605076, 76cc2...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, ..."
4,0002115f48f982,"[9ec225f0, 18281c6c, e3b6b115, 4a044c54, 365fe...",[0],"[1, 0, 0, 0, 0, 0, 0, 0, 0]"


In [8]:
tr_ids =   train_df["id"].unique()
train_order = order[order["id"].isin(tr_ids)]
val_order  = order[~(order["id"].isin(tr_ids))]
train_df = train_df.set_index("cell_id", drop=True)
val_df = val_df.set_index("cell_id", drop=True)

In [9]:
# train_order.head()
# val_order.head()
# train_df.head()

## test

In [10]:
#order.head()
#row = order.iloc[1]
#for cell_id in row.cell_shuffle:
#    print(cell_id)
#    print(train_df.loc[cell_id].source)
#    print("\n"*5)

In [11]:
#tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
#row = train_order.iloc[1]
#cells = row.cell_shuffle
#[train_df.loc[cell_id].source for cell_id in cells]
#inputs = tokenizer.batch_encode_plus(
#            [train_df.loc[cell_id].source for cell_id in cells],
#            add_special_tokens=True,
#            max_length=total_max_len,
#            padding="max_length",
#            # return_token_type_ids=True,
#            truncation=True
#        )
#ids = torch.LongTensor(inputs['input_ids'])
#mask = torch.LongTensor(inputs['attention_mask'])
#md_mask = torch.LongTensor(row.md_mask)
#permutation = torch.LongTensor(row.permutation)

        
#print("ids:", ids.size())
#print("mask:", mask.size())
#print("md_mask:", md_mask.size())

In [12]:
class MarkdownDataset(Dataset):
    def __init__(self, order_df, df, model_name_or_path, total_max_len, md_max_len, fts):
        super().__init__()
        self.order_df = order_df
        self.df = df
        self.md_max_len = md_max_len
        self.total_max_len = total_max_len  # maxlen allowed by model config
        self.tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
        self.fts = fts

    def __getitem__(self, index):
        row = self.order_df.iloc[index]
        cells = row.cell_shuffle

        # print("index: "+ str(index))

        inputs = self.tokenizer.batch_encode_plus(
            [str(self.df.loc[cell_id].source) for cell_id in cells],
            # None,
            add_special_tokens=True,
            max_length=self.total_max_len,
            padding="max_length",
            # return_token_type_ids=True,
            truncation=True
        )

        ids = torch.LongTensor(inputs['input_ids'])
        mask = torch.LongTensor(inputs['attention_mask'])
        md_mask = torch.LongTensor(row.md_mask)
        permutation = torch.LongTensor(row.permutation)
    
        length = len(md_mask)
        
        #print("ids:", ids.size())
        #print("mask:", mask.size())
        #print("md_mask:", md_mask.size())
        #return torch.FloatTensor([length])
        return ids, mask, permutation, md_mask, length

    def __len__(self):
        return self.order_df.shape[0]

In [13]:
train_ds = MarkdownDataset(train_order, train_df, model_name_or_path=model_name_or_path, md_max_len=md_max_len,
                           total_max_len=total_max_len, fts=train_fts)
val_ds = MarkdownDataset(val_order, val_df, model_name_or_path=model_name_or_path, md_max_len=md_max_len,
                         total_max_len=total_max_len, fts=val_fts)
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=n_workers,
                          pin_memory=False, drop_last=True)
val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False, num_workers=n_workers,
                        pin_memory=False, drop_last=False)

In [14]:
# model.cpu()
HIDDEN_SIZE = 768
class BersonModel(nn.Module):
    def __init__(self, model_path):
        super(BersonModel, self).__init__()
        self.model = AutoModel.from_pretrained(model_path)
        
        self.dropout = nn.Dropout(0.1)
        self.decoder = nn.LSTM(input_size=HIDDEN_SIZE, hidden_size=HIDDEN_SIZE, num_layers=1, batch_first=True)
        self.top = nn.Linear(HIDDEN_SIZE, 1)
        self.softmax = nn.Softmax(dim=0)
        self.layernorm = nn.LayerNorm(2)
        self.loss = nn.BCELoss()


    def forward(self, ids, mask, order, md_mask, length):
        """
        :param ids:
        :param mask:
        :param orders:
        :param type: 0 for code, 1 for md
        :return:
        """
        ids = ids.squeeze(0) # [:8,:]
        mask = mask.squeeze(0) # [:8,:]
        order = order.squeeze(0)
        md_mask = md_mask.squeeze(0)
        length = length.squeeze(0)
        # print("ids:", ids.size())
        #print("md_mask:", md_mask.size())
        #rint("length[:", length[0])
        
        
        #print(torch.cuda.memory_allocated(device=0) / (1024 * 1024))#0表示显卡号
        
        """
        print("size:",ids.size()[0])
        def get_batch():
            q = 16
            l = []
            while q < length+16:
                q_ids = ids[q-16:q, :]
                q_mask = mask[q-16:q, :]
                t = self.model(q_ids, q_mask)[1]
                q += 16
                l.append(t)
            return torch.cat(l, 0)
        """
        
        def get_batch():
            return self.model(ids, mask)[1].unsqueeze(0)
        
        if length > 64:
            with torch.no_grad():
                x = get_batch()
        else:
            x = get_batch()
        
        
        # hn -> (Layer, batch, hiddenstate)
        hn = torch.mean(x, 1).unsqueeze(1).cuda()
        cn = torch.zeros_like(hn).cuda()
        hcn = (hn, cn)
        
        md_len = len(order)
        # print("hn:", hn.size())
        # x = self.model(ids, mask)[1]
        # print(torch.cuda.memory_allocated(device=0) / (1024 * 1024))#0表示显卡号
        
        md_pos = [idx for idx, i in enumerate(md_mask) if i==1]
        # print("md_pos:", md_pos)
        # print("md_mask:", md_mask)
        # print("order:", order)
        md_pool = torch.swapaxes(x[:, md_pos, :], 0, 1)
        # print("md_num:",md_len)
        # print("md_pool:", md_pool.size())
        loss = 0
        j, idx = 0, 0
        for i in range(length):
            if md_mask[i] == 1:
                idx = order[j]
                j += 1
                # construct hcn for batch
                hn, cn = hcn
                hn = torch.tile(hn, (1, md_len, 1))
                cn = torch.tile(cn, (1, md_len, 1))
                t_hcn = (hn, cn)
                
                t_x, t_hcn = self.decoder(md_pool, t_hcn)
                # t_x: (n, 1, h)
                t_x = self.top(self.dropout(t_x))
                # print("t_x:", t_x.size(), "  idx:", idx)
                output = self.softmax(t_x).squeeze(2).squeeze(1)
                gt = torch.zeros_like(output)
                gt[idx] = 1.
                # print("output:",output)
                # print("gt:", gt)
                loss += self.loss(output, gt)
                t_hcn
            # print("test:", x[:, [i], :].size())
            output, hcn = self.decoder(x[:, [i], :], hcn)
        
                
        
        ## concat
        loss /= md_len
        # print("loss:", loss)

        # lstm
        #hn = 
        #hcn = 
        #for t in range():
        #    output, hcn = self.decoder(x, hcn) # hcn = (hidden_state, cell_state)
        # loss
        # loss = 1

        ## sentence order

        ## coherence
        return loss

In [15]:
import time
def read_data(d):
    return tuple([data.cuda() for data in d])


def validate(model, val_loader):
    model.eval()

    tbar = tqdm(val_loader, file=sys.stdout)

    preds = []
    labels = []

    with torch.no_grad():
        for idx, data in enumerate(tbar):
            inputs = read_data(data)

            with torch.cuda.amp.autocast():
                pred = model(*inputs)

            preds.append(pred.detach().cpu().numpy().ravel())
            labels.append(target.detach().cpu().numpy().ravel())

    return np.concatenate(labels), np.concatenate(preds)


def train(model, train_loader, val_loader, epochs):
    np.random.seed(0)
    # Creating optimizer and lr schedulers
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]

    num_train_optimization_steps = int(epochs * len(train_loader) / accumulation_steps)
    optimizer = AdamW(optimizer_grouped_parameters, lr=3e-5,
                      correct_bias=False)  # To reproduce BertAdam specific behavior set correct_bias=False
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0.05 * num_train_optimization_steps,
                                                num_training_steps=num_train_optimization_steps)  # PyTorch scheduler

    criterion = torch.nn.L1Loss()
    scaler = torch.cuda.amp.GradScaler()

    for e in range(epochs):
        model.train()
        tbar = tqdm(train_loader, file=sys.stdout)
        loss_list = []
        preds = []
        labels = []

        #if e <= 2:
            #y_val, y_pred = validate(model, val_loader)
            #val_df["pred"] = val_df.groupby(["id", "cell_type"])["rank"].rank(pct=True)
            #val_df.loc[val_df["cell_type"] == "markdown", "pred"] = y_pred
            #y_dummy = val_df.sort_values("pred").groupby('id')['cell_id'].apply(list)
            #print("Preds score", kendall_tau(df_orders.loc[y_dummy.index], y_dummy))
            #continue
        #print("epoch:", e)

        for idx, data in enumerate(tbar):
            inputs = read_data(data)
            target = 1
            
            time.sleep(3)
            with torch.cuda.amp.autocast():
                pred = model(*inputs)
                loss = criterion(pred, target)
            scaler.scale(loss).backward()
            if idx % args.accumulation_steps == 0 or idx == len(tbar) - 1:
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()
                scheduler.step()

            if idx % 10000 == 0:
                torch.save(model.state_dict(), f"./outputs/model_{idx}.bin")

            loss_list.append(loss.detach().cpu().item())
            preds.append(pred.detach().cpu().numpy().ravel())
            labels.append(target.detach().cpu().numpy().ravel())

            avg_loss = np.round(np.mean(loss_list), 4)
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
            tbar.set_description(f"Epoch {e + 1} Loss: {avg_loss} lr: {scheduler.get_last_lr()}")

        y_val, y_pred = validate(model, val_loader)
        val_df["pred"] = val_df.groupby(["id", "cell_type"])["rank"].rank(pct=True)
        val_df.loc[val_df["cell_type"] == "markdown", "pred"] = y_pred
        y_dummy = val_df.sort_values("pred").groupby('id')['cell_id'].apply(list)
        print("Preds score", kendall_tau(df_orders.loc[y_dummy.index], y_dummy))
        torch.save(model.state_dict(), f"./outputs/model_epoch_{e+1}.bin")

    return model, y_pred

model = BersonModel(model_name_or_path)
model = model.cuda()
model.load_state_dict(torch.load("./outputs/model_5000.bin"))

<All keys matched successfully>

In [16]:
# param_optimizer = list(model.named_parameters())
# print([n for n,p in param_optimizer])

In [17]:
np.random.seed(0)
# Creating optimizer and lr schedulers
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
add = ['decoder, top']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay+add)],'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in add)], "lr":3e-5 , 'weight_decay': 0.01},
]

num_train_optimization_steps = int(epochs * len(train_loader) / accumulation_steps)
print("steps:",num_train_optimization_steps, "acc_step:", accumulation_steps)
optimizer = AdamW(optimizer_grouped_parameters, lr=3e-5,
                  correct_bias=False)  # To reproduce BertAdam specific behavior set correct_bias=False
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0.05 * num_train_optimization_steps,
                                            num_training_steps=num_train_optimization_steps)  # PyTorch scheduler

criterion = torch.nn.L1Loss()
scaler = torch.cuda.amp.GradScaler()


steps: 250584 acc_step: 1


/home/mobius/anaconda3/envs/VIT/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
model.train()
tbar = tqdm(train_loader)
loss_list = []
preds = []
labels = []

#if e <= 2:
    #y_val, y_pred = validate(model, val_loader)
    #val_df["pred"] = val_df.groupby(["id", "cell_type"])["rank"].rank(pct=True)
    #val_df.loc[val_df["cell_type"] == "markdown", "pred"] = y_pred
    #y_dummy = val_df.sort_values("pred").groupby('id')['cell_id'].apply(list)
    #print("Preds score", kendall_tau(df_orders.loc[y_dummy.index], y_dummy))
    #continue
#print("epoch:", e)

avg_loss = 0
for idx, data in enumerate(tbar):
    # print(type(data), type(data[0]), data[0].size())
    # print("data[3].size()[1]: ",data[3].size()[1])
        
    data = read_data(data)
    
    #with torch.cuda.amp.autocast():
    loss = model(*data)
        # loss = criterion(pred, target)
    scaler.scale(loss).backward()
    if idx % accumulation_steps == 0 or idx == len(tbar) - 1:
        scaler.step(optimizer)
        scaler.update()
        optimizer.zero_grad()
        scheduler.step()

    if idx % 100 == 0:
        print("memory allocated:",torch.cuda.memory_allocated(device=0) / (1024 * 1024))
        print("avg_loss:", avg_loss)
    if idx % 5000 == 0:
        torch.save(model.state_dict(), f"./outputs/model_{idx}.bin")

    loss_list.append(loss.detach().cpu().item())
    # preds.append(pred.detach().cpu().numpy().ravel())
    # labels.append(target.detach().cpu().numpy().ravel())

    avg_loss = np.round(np.mean(loss_list[-1000:]), 4)

  0%|                                                                                                                                                                                                                                                                                                                                                                                             | 0/125292 [00:00<?, ?it/s]

memory allocated: 1986.9794921875
avg_loss: 0


  0%|▎                                                                                                                                                                                                                                                                                                                                                                               | 103/125292 [02:04<24:49:42,  1.40it/s]

memory allocated: 1987.001953125
avg_loss: 0.2528


  0%|▌                                                                                                                                                                                                                                                                                                                                                                               | 202/125292 [03:34<32:51:52,  1.06it/s]

...

  1%|██▎                                                                                                                                                                                                                                                                                                                                                                             | 802/125292 [12:59<16:45:02,  2.06it/s]

memory allocated: 1987.0087890625
avg_loss: 0.253


  1%|██▋                                                                                                                                                                                                                                                                                                                                                                            | 902/125292 [14:39<116:49:26,  3.38s/it]

memory allocated: 1987.015625
avg_loss: 0.2549


  1%|██▊                                                                                                                                                                                                                                                                                                                                                                             | 978/125292 [15:52<19:49:41,  1.74it/s]

## next

In [ ]:
with torch.cuda.amp.autocast():
        pred = model(*inputs)
        loss = criterion(pred, target)
    scaler.scale(loss).backward()
    if idx % args.accumulation_steps == 0 or idx == len(tbar) - 1:
        scaler.step(optimizer)
        scaler.update()
        optimizer.zero_grad()
        scheduler.step()

    if idx % 10000 == 0:
        torch.save(model.state_dict(), f"./outputs/model_{idx}.bin")

    loss_list.append(loss.detach().cpu().item())
    preds.append(pred.detach().cpu().numpy().ravel())
    labels.append(target.detach().cpu().numpy().ravel())

    avg_loss = np.round(np.mean(loss_list), 4)

    tbar.set_description(f"Epoch {e + 1} Loss: {avg_loss} lr: {scheduler.get_last_lr()}")



y_val, y_pred = validate(model, val_loader)
val_df["pred"] = val_df.groupby(["id", "cell_type"])["rank"].rank(pct=True)
val_df.loc[val_df["cell_type"] == "markdown", "pred"] = y_pred
y_dummy = val_df.sort_values("pred").groupby('id')['cell_id'].apply(list)
print("Preds score", kendall_tau(df_orders.loc[y_dummy.index], y_dummy))
torch.save(model.state_dict(), f"./outputs/model_epoch_{e+1}.bin")

In [ ]:
for e in range(epochs):
    model.train()
    tbar = tqdm(train_loader, file=sys.stdout)
    loss_list = []
    preds = []
    labels = []

    #if e <= 2:
        #y_val, y_pred = validate(model, val_loader)
        #val_df["pred"] = val_df.groupby(["id", "cell_type"])["rank"].rank(pct=True)
        #val_df.loc[val_df["cell_type"] == "markdown", "pred"] = y_pred
        #y_dummy = val_df.sort_values("pred").groupby('id')['cell_id'].apply(list)
        #print("Preds score", kendall_tau(df_orders.loc[y_dummy.index], y_dummy))
        #continue
    #print("epoch:", e)

    for idx, data in enumerate(tbar):
        inputs = read_data(data)
        target = 1

        time.sleep(3)
        with torch.cuda.amp.autocast():
            pred = model(*inputs)
            loss = criterion(pred, target)
        scaler.scale(loss).backward()
        if idx % args.accumulation_steps == 0 or idx == len(tbar) - 1:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
            scheduler.step()

        if idx % 10000 == 0:
            torch.save(model.state_dict(), f"./outputs/model_{idx}.bin")

        loss_list.append(loss.detach().cpu().item())
        preds.append(pred.detach().cpu().numpy().ravel())
        labels.append(target.detach().cpu().numpy().ravel())

        avg_loss = np.round(np.mean(loss_list), 4)

        tbar.set_description(f"Epoch {e + 1} Loss: {avg_loss} lr: {scheduler.get_last_lr()}")

    y_val, y_pred = validate(model, val_loader)
    val_df["pred"] = val_df.groupby(["id", "cell_type"])["rank"].rank(pct=True)
    val_df.loc[val_df["cell_type"] == "markdown", "pred"] = y_pred
    y_dummy = val_df.sort_values("pred").groupby('id')['cell_id'].apply(list)
    print("Preds score", kendall_tau(df_orders.loc[y_dummy.index], y_dummy))
    torch.save(model.state_dict(), f"./outputs/model_epoch_{e+1}.bin")

return model, y_pred

In [ ]:
# model.load_state_dict(torch.load("./outputs/model.bin"))
model, y_pred = train(model, train_loader, val_loader, epochs=epochs)


In [ ]:
train_df.loc['3a16a457']

In [28]:
s = time.time()
print(torch.cuda.memory_allocated(device=0) / (1024 * 1024))
print(time.time()-s)

2053.2802734375
0.0002880096435546875
